In [18]:
import json

def get_list2scrape(input, compare):
    with open(input, 'r') as f:
        input_list = json.load(f)
    with open(compare, 'r') as f:
        compare_list = json.load(f)

    compare_list = [element['url'] for element in compare_list]

    diff = list(set(input_list) - set(compare_list))

    return input_list, compare_list, diff

In [115]:
inp, comp, diff = get_list2scrape('links.json', 'test_parallel.json')

JSONDecodeError: Expecting value: line 1062514 column 1 (char 25655321)

In [116]:
len(inp)

4005

In [113]:
print(f'done: {len(comp)}')

done: 3994


In [114]:
print(f'left {len(diff)}')

left 11


In [110]:
diff

['https://www.parfumo.de/Parfums/the-dua-brand/Invasion_of_the_Barbers',
 'https://www.parfumo.de/Parfums/Rabanne/phantom-intense',
 'https://www.parfumo.de/Parfums/Lancaster/Le_Parfum_Solaire',
 'https://www.parfumo.de/Parfums/Issey_Miyake/L_Eau_d_Issey_pour_Homme_Eau_d_Ete_Summer_Edition_2014',
 'https://www.parfumo.de/Parfums/der-duft/cinematic',
 'https://www.parfumo.de/Parfums/Roja_Parfums/danger-parfum-cologne',
 'https://www.parfumo.de/Parfums/Salvador_Dali/Silver_Sun',
 'https://www.parfumo.de/Parfums/Roja_Parfums/enigma-parfum-cologne',
 'https://www.parfumo.de/Parfums/Roja_Parfums/51-essence-de-parfum',
 'https://www.parfumo.de/Parfums/Issey_Miyake/L_Eau_d_Issey_Eau_d_Ete_Summer_Edition_2016',
 'https://www.parfumo.de/Parfums/Roja_Parfums/elysium-parfum-cologne-eau-de-parfum',
 'https://www.parfumo.de/Parfums/Guerlain/Muguet_2013',
 'https://www.parfumo.de/Parfums/Issey_Miyake/L_Eau_d_Issey_pour_Homme_Eau_d_Ete_Summer_Edition_2015',
 'https://www.parfumo.de/Parfums/Estee_Laud

In [25]:
file_path = "short_list.json"
new_entries = [{"id": 6, "name": "Charlie"}, {"id": 7, "name": "David"}]

# Open file in append mode and modify JSON manually
with open(file_path, "r+", encoding="utf-8") as file:
    file.seek(0, 2)  # Move to the end of the file
    position = file.tell()
    
    if position > 2:  # If file is not empty, remove the last ']'
        file.seek(position - 2)
        file.truncate()
        file.write(",\n" + json.dumps(new_entries, indent=4)[1:])  # Append new items
    else:
        file.write(json.dumps(new_entries, indent=4)) 

In [ ]:
def save_results(self, results):
    """Save results to JSON file without overwriting existing data"""
    try:
        # Check if the file exists and is not empty
        if os.path.exists(self.output_file) and os.path.getsize(self.output_file) > 0:
            with open(self.output_file, "r+", encoding="utf-8") as f:
                try:
                    existing_data = json.load(f)  # Load existing data
                    if not isinstance(existing_data, list):  # Ensure it's a list
                        raise ValueError("JSON file does not contain a list.")
                except json.JSONDecodeError:
                    existing_data = []  # If corrupted, start fresh

                existing_data.extend(results)  # Append new results

                f.seek(0)  # Move cursor to the beginning
                json.dump(existing_data, f, ensure_ascii=False, indent=2)
                f.truncate()  # Remove leftover data if file shrinks
        else:
            with open(self.output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"Results saved to {self.output_file}")

    except Exception as e:
        print(f"Error saving results: {e}")

'臠'

In [123]:
dic =  [
                {
                    "r": "0",
                    "v": 0
                },
                {
                    "r": "1",
                    "v": 0
                },
                {
                    "r": "2",
                    "v": 0
                },
                {
                    "r": "3",
                    "v": 3
                },
                {
                    "r": "4",
                    "v": 9
                },
                {
                    "r": "5",
                    "v": 9
                },
                {
                    "r": "6",
                    "v": 18
                },
                {
                    "r": "7",
                    "v": 18
                },
                {
                    "r": "8",
                    "v": 4
                },
                {
                    "r": "9",
                    "v": 0
                },
                {
                    "r": "10",
                    "v": 1
                }
            ]

In [124]:
new_dic = {}
for element in dic:
    print(element)
    new_dic[element["r"]] = element["v"]

{'r': '0', 'v': 0}
{'r': '1', 'v': 0}
{'r': '2', 'v': 0}
{'r': '3', 'v': 3}
{'r': '4', 'v': 9}
{'r': '5', 'v': 9}
{'r': '6', 'v': 18}
{'r': '7', 'v': 18}
{'r': '8', 'v': 4}
{'r': '9', 'v': 0}
{'r': '10', 'v': 1}


In [125]:
new_dic

{'0': 0,
 '1': 0,
 '2': 0,
 '3': 3,
 '4': 9,
 '5': 9,
 '6': 18,
 '7': 18,
 '8': 4,
 '9': 0,
 '10': 1}

In [1]:
import json

In [118]:
with open('test_parallel.json', 'r', encoding='utf-8') as f:
    big_dic = json.load(f)


In [16]:
for i in range(1000):
    try:
        # print(big_dic[i]['detailed_ratings']['scent'][0])
        h = 3
    except:
        print(i)
        break

print(big_dic[222]['detailed_ratings'])

{'scent': None, 'durability': [{'r': '0', 'v': 1}, {'r': '1', 'v': 0}, {'r': '2', 'v': 0}, {'r': '3', 'v': 0}, {'r': '4', 'v': 0}, {'r': '5', 'v': 1}, {'r': '6', 'v': 2}, {'r': '7', 'v': 49}, {'r': '8', 'v': 34}, {'r': '9', 'v': 22}, {'r': '10', 'v': 30}], 'sillage': [{'r': '0', 'v': 0}, {'r': '1', 'v': 0}, {'r': '2', 'v': 1}, {'r': '3', 'v': 0}, {'r': '4', 'v': 0}, {'r': '5', 'v': 6}, {'r': '6', 'v': 4}, {'r': '7', 'v': 72}, {'r': '8', 'v': 31}, {'r': '9', 'v': 10}, {'r': '10', 'v': 16}], 'bottle': [{'r': '0', 'v': 0}, {'r': '1', 'v': 1}, {'r': '2', 'v': 6}, {'r': '3', 'v': 0}, {'r': '4', 'v': 0}, {'r': '5', 'v': 48}, {'r': '6', 'v': 24}, {'r': '7', 'v': 28}, {'r': '8', 'v': 13}, {'r': '9', 'v': 3}, {'r': '10', 'v': 4}], 'pricing': [{'r': '0', 'v': 0}, {'r': '1', 'v': 0}, {'r': '2', 'v': 0}, {'r': '3', 'v': 0}, {'r': '4', 'v': 0}, {'r': '5', 'v': 1}, {'r': '6', 'v': 0}, {'r': '7', 'v': 6}, {'r': '8', 'v': 2}, {'r': '9', 'v': 2}, {'r': '10', 'v': 0}]}


In [119]:
for idx, list_element in enumerate(big_dic):

    detailed_ratings = big_dic[idx]['detailed_ratings']

    for typ, rating_list in detailed_ratings.items():
        buf_dic = {}
        if rating_list is not None:
            for rating_element in rating_list:
                buf_dic[rating_element['r']] = rating_element['v']
            detailed_ratings[typ] = buf_dic
        else:
            detailed_ratings[typ] = None 


    big_dic[idx]['detailed_ratings'] = detailed_ratings

In [120]:
with open('corrected_perfumes.json', 'w', encoding='utf-8') as file:
    json.dump(big_dic, file, indent=2)